In [3]:
from models import ICNet
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the ICNet model with the same configuration as during training
net = ICNet(num_class=19, n_channel=3, backbone_type='resnet18', act_type='relu', use_aux=True)

# Load the checkpoint
checkpoint_path = 'C:/Users/avshinde/Downloads/New folder/New folder/icnet.pth'  # Replace with the actual path to your checkpoint file
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Add map_location argument if needed
print(checkpoint.keys())

# Load the model state_dict
net.load_state_dict(checkpoint['state_dict'])

net=net.to(device)

# Ensure the model is in evaluation mode
net.eval()

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


dict_keys(['cur_epoch', 'best_score', 'state_dict', 'optimizer', 'scheduler'])


ICNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [3]:
total_params = 0
non_zero_params = 0
zero_params = 0

for name, param in net.state_dict().items():
        total_params += param.numel()
        non_zero_params += torch.count_nonzero(param).item()
        zero_params += (param.numel() - torch.count_nonzero(param)).item()
print(f"Total Parameters: {total_params}")
print(f"Non-Zero Parameters: {non_zero_params}")
print(f"Zero Parameters: {zero_params}")

Total Parameters: 12881215
Non-Zero Parameters: 12881215
Zero Parameters: 0


### Prune

In [4]:
from prune import prune_icnet
prune_icnet(net, method='std', s=0.25)

In [5]:
# If needed, remove pruning
from torch.nn.utils import prune

for name, module in net.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        prune.remove(module, 'weight')

In [6]:
total_params = 0
non_zero_params = 0
zero_params = 0

for name, param in net.state_dict().items():
        total_params += param.numel()
        non_zero_params += torch.count_nonzero(param).item()
        zero_params += (param.numel() - torch.count_nonzero(param)).item()
print(f"Total Parameters: {total_params}")
print(f"Non-Zero Parameters: {non_zero_params}")
print(f"Zero Parameters: {zero_params}")

Total Parameters: 12881215
Non-Zero Parameters: 9666239
Zero Parameters: 3214976


In [7]:
# # Print the names and values of the parameters
# for name, param in net.named_parameters():
#     print(f'Parameter name: {name}')
#     print(f'Parameter shape: {param.shape}')
#     print(f'Parameter data:\n{param.data}\n')

In [7]:
# Create a new model to store the pruned weights
pruned_model = ICNet(num_class=19, n_channel=3, backbone_type='resnet18', act_type='relu', use_aux=True)
pruned_model.load_state_dict(net.state_dict())

<All keys matched successfully>

In [8]:
# Save the updated checkpoint
updated_checkpoint_path = 'updated_checkpoint_icnet.pth'
checkpoint['state_dict'] = pruned_model.state_dict()
checkpoint['cur_epoch'] = 0
checkpoint['best_score'] = 0
torch.save(checkpoint, updated_checkpoint_path)

### Fine-tuning pruned weights and prediction

##### Original mIoU: 69.65


##### Obtained mIoU: 66.18

In [ ]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 11:32] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/updated_checkpoint_icnet.pth
[2023-12-04 11:32] Resume training from C:/Users/avshinde/Downloads/New folder/New folder/updated_checkpoint_icnet.pth
[2023-12-04 11:32] 


######################### Config Informations #########################
dataset: cityscapes
num_class: 19
model: icnet
encoder: None
decoder: None
loss_type: ohem
optimizer_type: adam
lr_policy: cos_warmup
total_epoch: 10
train_bs: 8
val_bs: 8
train_num: 2968
val_num: 500
gpu_num: 1
num_workers: 8
amp_training: False
DDP: False
kd_training: False
synBN: True
use_ema: True
use_aux: True
#######################################################################


E

tensor(0.4977, device='cuda:0')


Epoch:2/10    |Loss:2.885    |: 100%|██████████| 371/371 [05:30<00:00,  1.12it/s]
Validating:    |: 100%|██████████| 63/63 [02:18<00:00,  2.19s/it] 
[2023-12-04 11:47]  Epoch2 mIoU: 0.5912    | best mIoU so far: 0.4977



tensor(0.5912, device='cuda:0')


Epoch:3/10    |Loss:4.653    |: 100%|██████████| 371/371 [05:32<00:00,  1.11it/s]
Validating:    |: 100%|██████████| 63/63 [02:15<00:00,  2.15s/it] 
[2023-12-04 11:55]  Epoch3 mIoU: 0.6196    | best mIoU so far: 0.5912



...

tensor(0.6462, device='cuda:0')


Epoch:7/10    |Loss:3.123    |: 100%|██████████| 371/371 [05:34<00:00,  1.11it/s]
Validating:    |: 100%|██████████| 63/63 [02:18<00:00,  2.20s/it] 
[2023-12-04 12:27]  Epoch7 mIoU: 0.6532    | best mIoU so far: 0.6462



tensor(0.6532, device='cuda:0')


  0%|          | 0/371 [00:14<?, ?it/s]


KeyboardInterrupt: 

### Predicting on the fine-tuned weights

In [2]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

[2023-12-04 12:33] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/save/last.pth
[2023-12-04 12:33] 
Start predicting...

100%|██████████| 38/38 [09:22<00:00, 14.81s/it]


### Video Output

In [1]:
import cv2
import os
from natsort import natsorted

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save/mainz_finetuned/blend'

# Output video file name
video_name = 'output_video_mainz_finetuned.avi'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith("blend.png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 1 # Frames per second

# Create a VideoWriter object with the desired frame rate
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), desired_frame_rate, (width, height))

# Loop through the images and write each frame to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the VideoWriter object
video.release()

In [3]:
import cv2
import os
from natsort import natsorted
import imageio

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save'

# Output GIF file name
gif_name = 'output_video_mainz.gif'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith("blend.png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 0.7 # Frames per second

# Create a list to store frames
frames = []

# Loop through the images and append each frame to the list
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    # Ensure that the color channels are in the correct order (BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

# Save the frames as a GIF using imageio
imageio.mimsave(gif_name, frames, duration=1/desired_frame_rate)

# Note: The duration parameter specifies the time each frame is displayed in seconds (1/desired_frame_rate).

print(f'GIF saved as {gif_name}')


GIF saved as output_video_mainz.gif


In [4]:
# Define the ICNet model with the same configuration as during training
model_1 = ICNet(num_class=19, n_channel=3, backbone_type='resnet18', act_type='relu', use_aux=True)

# Load the checkpoint
checkpoint_path = 'C:/Users/avshinde/Downloads/New folder/New folder/save/last.pth'  # Replace with the actual path to your checkpoint file
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Add map_location argument if needed
print(checkpoint.keys())

# Load the model state_dict
model_1.load_state_dict(checkpoint['state_dict'])

model_1=model_1.to(device)

# Ensure the model is in evaluation mode
model_1.eval()

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


dict_keys(['cur_epoch', 'best_score', 'state_dict', 'optimizer', 'scheduler'])


ICNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [5]:
total_params = 0
non_zero_params = 0
zero_params = 0

for name, param in model_1.state_dict().items():
        total_params += param.numel()
        non_zero_params += torch.count_nonzero(param).item()
        zero_params += (param.numel() - torch.count_nonzero(param)).item()
print(f"Total Parameters: {total_params}")
print(f"Non-Zero Parameters: {non_zero_params}")
print(f"Zero Parameters: {zero_params}")

Total Parameters: 12881215
Non-Zero Parameters: 11430304
Zero Parameters: 1450911


## Validating the original model

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 09:49] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/icnet.pth
[2023-12-04 09:49] Resume training from C:/Users/avshinde/Downloads/New folder/New folder/icnet.pth
[2023-12-04 09:49] 


######################### Config Informations #########################
dataset: cityscapes
num_class: 19
model: icnet
encoder: None
decoder: None
loss_type: ohem
optimizer_type: adam
lr_policy: cos_warmup
total_epoch: 10
train_bs: 8
val_bs: 8
train_num: 2968
val_num: 500
gpu_num: 1
num_workers: 8
amp_training: False
DDP: False
kd_training: False
synBN: True
use_ema: True
use_aux: True
#######################################################################


[2023-12-04 09:49] 
Train 10 epochs fin

tensor(0.6965, device='cuda:0')


##### mIoU: 69.95

### Predicting on the original model

In [1]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-12-04 09:57] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/icnet.pth
[2023-12-04 09:57] 
Start predicting...

100%|██████████| 38/38 [09:29<00:00, 14.97s/it]


#### Video Output

In [2]:
import cv2
import os
from natsort import natsorted

# Directory containing the predicted segmentation images
image_folder = 'C:/Users/avshinde/Downloads/New folder/New folder/save/mainz_unpruned/blend'

# Output video file name
video_name = 'output_video_mainz_unpruned.avi'

# Get the list of image files and sort them
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images = natsorted(images)

# Get image dimensions from the first image
img = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = img.shape

# Define the desired frame rate (adjust as needed)
desired_frame_rate = 1 # Frames per second

# Create a VideoWriter object with the desired frame rate
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), desired_frame_rate, (width, height))

# Loop through the images and write each frame to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the VideoWriter object
video.release()

In [2]:
# Define the ICNet model with the same configuration as during training
net = ICNet(num_class=19, n_channel=3, backbone_type='resnet18', act_type='relu', use_aux=True)

# Load the checkpoint
checkpoint_path = 'C:/Users/avshinde/Downloads/New folder/New folder/last.pth'  # Replace with the actual path to your checkpoint file
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Add map_location argument if needed
print(checkpoint.keys())

C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\avshinde\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


dict_keys(['cur_epoch', 'best_score', 'state_dict', 'optimizer', 'scheduler'])


In [3]:
from core import SegTrainer
from configs import MyConfig, load_parser

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    config = MyConfig()
    
    config.init_dependent_config()

    #config.model = net
    # If you want to use command-line arguments, please uncomment the following line
    # config = load_parser(config)

    trainer = SegTrainer(config)
    
    if config.is_testing:
        trainer.predict(config)
    else:    
        trainer.run(config)

[2023-12-04 11:08] Load model state dict from C:/Users/avshinde/Downloads/New folder/New folder/last.pth
[2023-12-04 11:08] Resume training from C:/Users/avshinde/Downloads/New folder/New folder/last.pth
[2023-12-04 11:08] 


######################### Config Informations #########################
dataset: cityscapes
num_class: 19
model: icnet
encoder: None
decoder: None
loss_type: ohem
optimizer_type: adam
lr_policy: cos_warmup
total_epoch: 10
train_bs: 8
val_bs: 8
train_num: 2968
val_num: 500
gpu_num: 1
num_workers: 8
amp_training: False
DDP: False
kd_training: False
synBN: True
use_ema: True
use_aux: True
#######################################################################


[2023-12-04 11:08] 
Train 10 epochs finished!

[2023-12-04 11:08] ##################################################
Validation for the best checkpoint...
Validating:    |: 100%|██████████| 63/63 [02:14<00:00,  2.14s/it] 
[2023-12-04 11:11] 

Train 10 epochs finished.

Best mIoU is: 0.0356

[2023-12-04 11:11] 

tensor(0.0356, device='cuda:0')
